# Neurosnap API Tutorial
[![Neurosnap Header](https://raw.githubusercontent.com/NeurosnapInc/neurosnap/refs/heads/main/assets/header.webp)](https://neurosnap.ai/)

In this tutorial, we demonstrate how to use the NeurosnapAPI class and associated methods from the [Neurosnap SDK package](https://github.com/NeurosnapInc/neurosnap/). This guide will walk you through the various functionalities provided by the neurosnap API, helping you understand how to interact with neurosnap services, manage jobs, and work with team information.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NeurosnapInc/neurosnap/blob/main/example_notebooks/api.ipynb)

#### Part 0. Installing Dependencies on Colab

In [1]:
import subprocess
import sys

if 'google.colab' in sys.modules:
    subprocess.run("pip install -U --no-cache-dir git+https://github.com/NeurosnapInc/neurosnap.git", shell=True, check=True, executable="/bin/bash")

#### Part 1. `NeurosnapAPI` Class: Connecting to neurosnap
The `NeurosnapAPI` class allows you to connect to the neurosnap platform using your API key. Upon creating an instance of this class, it automatically checks the validity of your key by making a request to the neurosnap API. If the key is invalid, an error will be raised, prompting you to verify your credentials. This authentication step is essential for accessing the platform’s features, such as submitting jobs and retrieving data.
You can generate and delete API keys by visiting the [neurosnap Overview page](https://neurosnap.ai/overview) under the "API Key" section.

In [2]:
import os

import pandas as pd
from neurosnap.api import NeurosnapAPI
from neurosnap.protein import Protein

api_key = os.environ.get('ns_key')
api = NeurosnapAPI(api_key) # replace with your API key


[*] ⚠️ NEVER SHARE YOUR API KEY WITH ANY UNAUTHORIZED PARTIES ⚠️
[*] Successfully connected to the Neurosnap API.
 - For information visit https://neurosnap.ai/blog/post/66b00dacec3f2aa9b4be703a
 - For support visit https://neurosnap.ai/support
 - For bug reports visit https://github.com/NeurosnapInc/neurosnap


#### Part 2. `get_services` Method: Retrieving neurosnap services
The get_services method allows you to fetch a list of all available services on the neurosnap platform. This function gives you the flexibility to print the services in different formats or simply retrieve them for further use.

Functionality:
- Fetch Services: It retrieves a list of available services from neurosnap's API and returns it as a list of dictionaries.
- Optional Printing: You can optionally print the services in two formats:
    - "table": Displays services in a table format showing key information (title, beta status, link).
    - "json": Prints the raw JSON response, formatted for readability.
    - None: By default, the function will not print anything but will return the data.

In [3]:
services = api.get_services(format_type="table")

Title                                         Beta    Link
--------------------------------------------  ------  -------------------------------------------------------------------------------
NeuroBind                                     False   https://neurosnap.ai/service/NeuroBind
NeuroFold                                     False   https://neurosnap.ai/service/NeuroFold
Boltz-2 (AlphaFold3)                          False   https://neurosnap.ai/service/Boltz-2%20%28AlphaFold3%29
Boltz-1 (AlphaFold3)                          False   https://neurosnap.ai/service/Boltz-1%20%28AlphaFold3%29
Chai-1 (AlphaFold3)                           False   https://neurosnap.ai/service/Chai-1%20%28AlphaFold3%29
Protenix (AlphaFold3)                         False   https://neurosnap.ai/service/Protenix%20%28AlphaFold3%29
IntelliFold (AlphaFold3)                      False   https://neurosnap.ai/service/IntelliFold%20%28AlphaFold3%29
AlphaFold2                                    False   https://neuro

#### Part 3. Job Management Methods in NeurosnapAPI
The following methods allow you to manage and interact with jobs submitted to neurosnap. These methods cover everything from submitting new jobs, checking job status, and retrieving job files, to managing notes and sharing functionality.

`get_jobs`: Retrieve submitted jobs

This method fetches a list of all submitted jobs from neurosnap.

- Fetch Jobs: Retrieves submitted jobs as a list of dictionaries.
- Optional Printing:
    - "table": Displays jobs in a table format.
    - "json": Prints the jobs in formatted JSON.
    - None: No printing by default, just returns the job data.

In [4]:
jobs = api.get_jobs(format_type="table")

Status     Job ID                    User ID                   Service                          Submitted    Runtime      Credits Used  Note                               Shared                     Team ID               Pipeline ID    Pipeline Node ID    Pipeline Item Index
---------  ------------------------  ------------------------  -------------------------------  -----------  ---------  --------------  ---------------------------------  --------  ------------------------  ------------------------  ------------------  ---------------------
pending    68e2ba96e986d44f8b7e9ff5  66c61493dfaceccb0cca0af1  ProteinMPNN                      2025-10-05   N/A             0          My ProteinMPNN job for A0A411PAU0  False     000000000000000000000000  000000000000000000000000                   0                      0
pending    68e2b83ee986d44f8b7e9ff4  66c61493dfaceccb0cca0af1  ProteinMPNN                      2025-10-05   N/A             0          My ProteinMPNN job for A0A411PAU0  Fals

`submit_job`: Submit a new job

This method submits a new job to neurosnap.

- Service Name: The neurosnap service to run.
- Files: A dictionary mapping file names to file paths.
- Config: Any parameters required by the service.
- Job ID: Returns the unique job id upon successful submission.

For this example, we will use the ProteinMPNN service. To ensure proper file and configuration setup for the service you intend to use, we recommend checking the service page on neurosnap for detailed information about required inputs.

For instance, the [ProteinMPNN service page](https://neurosnap.ai/service/ProteinMPNN) specifies required files and configuration parameters, such as input structure, number of sequences, and sampling temperature.

In [5]:
prot = Protein("A0A411PAU0")  # loading a protein
prot.save("A0A411PAU0.pdb")  # saving the protein structure to a PDB file
# Checkbox fields should not be sent if they are false
fields = {
  "Input Structure": ("structure.pdb", open("A0A411PAU0.pdb", "rb")),
  #"Homo-oligomer": "false",
  "Fixed Positions": "",
  #"Invert Selection": "false",
  "Number Sequences": "100",
  "Sampling Temperature": "0.15",
  "Model Type": "v_48_020",
  "Model Version": "original",
}
job_id = api.submit_job("ProteinMPNN", fields=fields, note="My ProteinMPNN job for A0A411PAU0")

[*] Found matching structure in AF-DB, downloading and using that.
[*] ProteinMPNN job successfully submitted with ID 68e2ce1ce986d44f8b7ea000.


`get_job_status`: Check Job Status

This method retrieves the status of a job by its id.

- Job ID: Provide the id of the job.
- Status Retrieval: Returns the current job status (e.g., "pending," "running," "completed").

In [6]:
api.get_job_status(job_id)

'completed'

`get_job_data`: Retrieve config data and files from a neurosnap job.

In [7]:
api.get_job_data(job_id)

{'config': {'Alanine Bias': 0,
  'Arginine Bias': 0,
  'Asparagine Bias': 0,
  'Aspartic acid Bias': 0,
  'Cysteine Bias': 0,
  'Fixed Positions': '',
  'Glutamic acid Bias': 0,
  'Glutamine Bias': 0,
  'Glycine Bias': 0,
  'Histidine Bias': 0,
  'Homo-oligomer': False,
  'Input Structure': 'Input_Structure.zip',
  'Invert Selection': False,
  'Isoleucine Bias': 0,
  'Leucine Bias': 0,
  'Lysine Bias': 0,
  'Methionine Bias': 0,
  'Model Type': 'v_48_020',
  'Model Version': 'original',
  'Number Sequences': 100,
  'Phenylalanine Bias': 0,
  'Proline Bias': 0,
  'Sampling Temperature': 0.15000000596046448,
  'Serine Bias': 0,
  'Threonine Bias': 0,
  'Tryptophan Bias': 0,
  'Tyrosine Bias': 0,
  'Valine Bias': 0},
 'in': [['Input_Structure.zip', '17.54 KB']],
 'out': [['pssm_conditional.json', '56.64 KB'],
  ['pssm_unconditional.json', '56.58 KB'],
  ['results.csv', '13.96 KB']]}

`get_job_file`: Download a specific file.

This method allows you to download a specific file from a job.

- Job ID & File Name: Provide the job id, file type, and file name.
- Save Path: Specify where the file should be saved locally.
- Optional Share ID: If the job is shared, include the share id. (for shared jobs only that are not submitted by you) 

In [8]:
api.get_job_file(job_id, 'out', 'results.csv', save_path='./results.csv')

[*] File saved to ./results.csv


For example, the `results.csv` file is the results of our ProteinMPNN job.

In [9]:
df = pd.read_csv('results.csv')
display(df.head())

,rank,score,seqid,seq
0,1,0.804,0.394,SVTLTESGGGTVAAGGSTTLTCTASKSAFNSSSATWAWVAQYPGGP...
1,2,0.808,0.394,AVTLTESGGGTVAAGGSVTLTCTASSPAFNSSTATWAWLAQYAGGP...
2,3,0.814,0.378,EVVLTESGGGTVAAGGSTTLTCTASSPVFNSSSATWAWVAQKPGGP...
3,4,0.814,0.394,EVVLTESGGGTVAEGGSVTLTCKASKSVFNSSSATWSWLAQKAGGA...
4,5,0.824,0.433,TITLTESGGGTVAAGGSTTLTCTASVSAFTGSTDTWAWWAQKPGGP...


`set_job_note`: Add a note to a job

This method lets you attach a note to a job for easier management.

- Job ID: Specify the job to attach the note to.
- Note: The text note you want to add.

In [10]:
api.set_job_note(job_id, 'Hello World!')

[*] Note set successfully for job ID 68e2ce1ce986d44f8b7ea000.


`set_job_share`: Enable Job Sharing

This method makes a job publicly accessible by enabling sharing.

- Job ID: Provide the job id to enable sharing.
- Share ID: The method returns the share id, which can be used to access the job publicly.

In [11]:
share_id = api.set_job_share(job_id)

print(f"https://neurosnap.ai/job/{job_id}?share={share_id}") # Public link to the job

[*] Job with ID 68e2ce1ce986d44f8b7ea000 is now public with share ID 68e2d123e986d44f8b7ea002.


https://neurosnap.ai/job/68e2ce1ce986d44f8b7ea000?share=68e2d123e986d44f8b7ea002


`delete_job_share`: Disable job sharing

This method disables the sharing feature and makes a job private.

- Job ID: Provide the job id to make it private.
- Action: The job is no longer publicly accessible.

In [12]:
api.delete_job_share(job_id)

[*] Job with ID 68e2ce1ce986d44f8b7ea000 is now private.


#### Part 4. Team Collaboration
The following methods allow you to manage and retrieve information about your neurosnap team, including team details and jobs submitted by team members.

`get_team_info`: Fetch Team Information
This method retrieves your team's information if you are part of a neurosnap Team.

- Format Type: Optional parameter to specify the output format. You can choose from:
    - table: Displays team information in a tabular format.
    - json: Displays team information in JSON format.
    - None: No printing by default, just returns the data.
- Returns: A dictionary containing the team information.

In [ ]:
team_info = api.get_team_info(format_type="table")

`get_team_jobs`: Fetch Team Jobs
This method retrieves all the jobs submitted by all members of your neurosnap Team.

- Format Type: Optional parameter to specify the output format. You can choose from:
    - table: Displays the list of jobs in a tabular format.
    - json: Displays the list of jobs in JSON format.
    - None: No printing by default, just returns the data.
- Returns: A list of dictionaries representing jobs submitted by team members. 

In [ ]:
team_jobs = api.get_team_jobs(format_type="table")